### Example 1: allocate to fastest resource

In this simple example we have 2 tasks which require the same resource. Task 2 depends on Task 1


In [3]:
from factryengine import Task, Resource, Scheduler, Assignment, ResourceGroup

# create the resource
resource = Resource(id=1, available_windows=[(0, 10), (20, 30)])

# create the resource group
resource_group = ResourceGroup(resources=[resource])

# create the assignment
assignment = Assignment(resource_groups=[resource_group], resource_count=1)

# create tasks
t1 = Task(id=1, duration=5, priority=1, constraints=[resource], predecessor_ids=[2])
t2 = Task(id=2, duration=5, priority=1, constraints=[resource])

tasks = [t1, t2]
result = Scheduler(tasks=tasks, resources=[resource]).schedule()

Scheduled 2 of 2 tasks.


In [4]:
# we can get the result as a dataframe
result.to_dataframe()

,task_id,assigned_resource_ids,task_start,task_end,resource_intervals
0,1,[1],5,10,"((5, 10))"
1,2,[1],0,5,"((0, 5))"


Task 2 is scheduled before task 1 because task 2 is a predecessor of task 1.


### Example 2: allocate to fastest team


In [5]:
r1 = Resource(id=1, available_windows=[(2, 10), (20, 30)])
r2 = Resource(id=2, available_windows=[(2, 10), (20, 30)])
r3 = Resource(id=3, available_windows=[(0, 10), (20, 30)])
r4 = Resource(id=4, available_windows=[(0, 10), (20, 30)])

team1 = ResourceGroup(resources=[r1, r2])
team2 = ResourceGroup(resources=[r3, r4])

assignment1 = Assignment(resource_groups=[team1, team2], use_all_resources=True)

t1 = Task(id=1, duration=6, assignments=[assignment1], priority=1)

tasks = [t1]
resources = [r1, r2, r3, r4]

result = Scheduler(tasks=tasks, resources=resources).schedule()
result.to_dataframe()

Scheduled 1 of 1 tasks.


,task_id,assigned_resource_ids,task_start,task_end,resource_intervals
0,1,"[3, 4]",0,3,"((0, 3), (0, 3))"


duration of 6 is cut in half due to there being 2 workers.


### Example 3: Combining Constraint and assignments

Scenario could be you have a task that require a machine which should be there for the entire duration of the task, and you have a group of resources which can operate the machine but you only need one.


In [6]:
machine = Resource(id=1, available_windows=[(20, 30)])
operator1 = Resource(id=2, available_windows=[(0, 10), (20, 28)])
operator2 = Resource(id=3, available_windows=[(15, 25)])
operator3 = Resource(id=4, available_windows=[(0, 10), (20, 30)])

operator_group = ResourceGroup(resources=[operator1, operator2, operator3])

assignment = Assignment(resource_groups=[operator_group], resource_count=1)

# add machine as a constraint
t1 = Task(
    id=1, duration=10, assignments=[assignment], priority=1, constraints=[machine]
)

tasks = [t1]
resources = [operator1, operator2, operator3, machine]

result = Scheduler(tasks=tasks, resources=resources).schedule()
result.to_dataframe()

Scheduled 1 of 1 tasks.


,task_id,assigned_resource_ids,task_start,task_end,resource_intervals
0,1,"[4, 1]",20,30,"((20, 30), (20, 30))"


we can see that the task is scheduled to start at 20 and end at 30, which is the only time when the machine is available. Operator with id 4 is selected as he is the only one avaialble.
